In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

## Dummy Cleaning

In [3]:
df = pd.read_csv('Dummy.csv')

In [7]:
Train = pd.read_csv("Train_Set.csv")
Train.drop(['Unnamed: 0'], axis=1, inplace=True)
Train.drop(['zip_code_A>'], axis=1, inplace=True)

int64_cols = Train.shape[1] - 6
cols_convert = Train.columns[:int64_cols]
dtype_dict = {col: np.int8 for col in cols_convert}
Train[cols_convert] = Train[cols_convert].astype(dtype_dict)

In [18]:
copy = Train
copy = copy.iloc[[]]

In [ ]:
copy.drop(['binned_label'], axis=1, inplace=True)

In [ ]:
copy['bed'] = df['bed']
copy['bath'] = df['bath']
copy['acre_lot'] = df['acre_lot']
copy['house_size'] = df['house_size']
copy['label'] = df['price'].round(0)

In [ ]:
copy = copy.reset_index(drop=True)

In [ ]:
New_DF = copy.iloc[:12]

In [26]:
New_DF.to_csv("Final_Dummy.csv")

## Making the Train Set

In [9]:
df = pd.read_csv('Old_Dataset.csv')

In [10]:
city_counts = df['city'].value_counts()
df = df[df['city'].isin(city_counts[city_counts >= 1000].index)]

In [11]:
df["bed"] = df["bed"].convert_dtypes(int)
df["bath"] = df["bath"].convert_dtypes(int)
df["zip_code"] = df["zip_code"].convert_dtypes(int)

df['zip_code'] = df['zip_code'].astype(str).fillna('')
df['zip_code'] = df['zip_code'].apply(lambda x: x.zfill(5)) 

In [12]:
df['zip_code'] = df['zip_code'].replace("<NA>", "100")
df['zip_code'] = df['zip_code'].str[3:]

df['sold'] = df['prev_sold_date'].notnull().map({True: 'yes', False: 'no'})
df.drop(["prev_sold_date"], axis=1, inplace=True)

In [13]:
df.dropna(subset=["price"], axis = 0, inplace=True)
df.drop(df[df["price"] <= 1000].index, axis = 0, inplace=True)

n_bins = 50
quantiles = df['price'].quantile(np.linspace(0, 1, n_bins + 1)[1:])
min_price = df['price'].min() 

bins = quantiles.tolist()
bins.insert(0, min_price)

price_categories = pd.cut(df['price'], bins=bins, include_lowest=True)
df['price_category'] = price_categories

price_category_bed_means = df.groupby('price_category')['bed'].mean().astype(int)
price_category_bath_means = df.groupby('price_category')['bath'].mean().astype(int)
price_category_size_means = df.groupby('price_category')['house_size'].mean()
price_category_acre_means = df.groupby('price_category')['acre_lot'].mean()


def fill_bed(row):
    category = row['price_category']
    if pd.isna(row['bed']):
        return price_category_bed_means.loc[category]
    else:
        return row['bed']


df["bed"] = df.apply(fill_bed, axis=1)


def fill_bath(row):
    category = row['price_category']
    if pd.isna(row['bath']):
        return price_category_bath_means.loc[category]
    else:
        return row['bath']


df["bath"] = df.apply(fill_bath, axis=1)

n_bins = 10

bins2 = [1, 2, 3, 4, 5, 6, 10, 15, 25, 142]
bed_categories = pd.cut(df['bed'], bins=bins2, include_lowest=True)
df['bed_category'] = bed_categories

bed_category_size_means = df.groupby('bed_category')['house_size'].mean()
bed_category_acre_means = df.groupby('bed_category')['acre_lot'].mean()

bins3 = [1, 2, 3, 4, 5, 6, 10, 15, 25, 198]
bath_categories = pd.cut(df['bath'], bins=bins3, include_lowest=True)
df['bath_category'] = bath_categories

bath_category_size_means = df.groupby('bath_category')['house_size'].mean()
bath_category_acre_means = df.groupby('bath_category')['acre_lot'].mean()


def fill_house_size(row):
    category1 = row['price_category']
    category2 = row['bed_category']
    category3 = row['bath_category']
    if pd.isna(row['house_size']):
        return (price_category_size_means.loc[category1] + bed_category_size_means.loc[category2] +
                bath_category_size_means.loc[category3]) / 3
    else:
        return row['house_size']


df["house_size"] = df.apply(fill_house_size, axis=1)


def fill_acre_lot(row):
    category1 = row['price_category']
    category2 = row['bed_category']
    category3 = row['bath_category']
    if pd.isna(row['acre_lot']):
        return (price_category_acre_means.loc[category1] + bed_category_acre_means.loc[category2] +
                bath_category_acre_means.loc[category3]) / 3
    else:
        return row['acre_lot']


df["acre_lot"] = df.apply(fill_acre_lot, axis=1)

df.drop(["price_category"], axis=1, inplace=True)
df.drop(["bed_category"], axis=1, inplace=True)
df.drop(["bath_category"], axis=1, inplace=True)

/var/folders/j9/nrpllg2s5fx8xthz6hx8jr6h0000gn/T/ipykernel_7521/1582166319.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_category_bed_means = df.groupby('price_category')['bed'].mean().astype(int)
/var/folders/j9/nrpllg2s5fx8xthz6hx8jr6h0000gn/T/ipykernel_7521/1582166319.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_category_bath_means = df.groupby('price_category')['bath'].mean().astype(int)
/var/folders/j9/nrpllg2s5fx8xthz6hx8jr6h0000gn/T/ipykernel_7521/1582166319.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pa

In [14]:
def create_bins(df):
    quantiles = df.quantile([0, 0.1, 0.25, 0.5, 0.75, 0.9, 1])
    bins_df = pd.cut(df, bins=quantiles,
                     labels=["1k - 89k", "90k - 187k", "188k - 373k", "374k - 699k", "700k - 1424k", "1425k - 875000k"])
    bins_df = bins_df.fillna("1k - 89k")
    return bins_df


features = df.drop(columns=['price'])
label = df["price"]

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

y_train_binned = create_bins(y_train)
Train = X_train
Train['binned_label'] = y_train_binned

Train.to_csv("Final_Train_Set.csv")

## Adding the train data to the database

In [16]:
db_user = "root"
db_password = "Heggi_2002"
db_host = "localhost"
db_port = "3306"
db_name = "Home_Scout"

engine_str = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(engine_str)

Train.to_sql('Train', con=engine, index=False)
engine.dispose()